In [1]:
# Example for making a basic candlestick plot.

# Imports
import pandas as pd
import plotly.graph_objects as go
# only need this import to make Jupyter show the graph in PyCharm:
import plotly.io as pio

# 1) Read a CSV from URL
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

# Print the column names so that you know what to use for the candlestick plot
print(df.columns)

# 2) Make the candlestick
fig = go.Figure(
    data=[
        go.Candlestick(
            x=df['Date'],
            open=df['AAPL.Open'],
            high=df['AAPL.High'],
            low=df['AAPL.Low'],
            close=df['AAPL.Close']
        )
    ]
)

fig

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1108)>

In [2]:
from ibapi.contract import Contract
from fintech_ibkr import *

import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import plotly.graph_objects as go

class ibkr_app(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.error_messages = pd.DataFrame(columns=[
            'reqId', 'errorCode', 'errorString'
        ])
        self.next_valid_id = None
        self.historical_data = pd.DataFrame(columns=["date", "open", "high", "low", "close"])
        self.historical_data_end = ''

    def error(self, reqId, errorCode, errorString):
        print("Error: ", reqId, " ", errorCode, " ", errorString)

    def nextValidId(self, orderId: int):
        self.next_valid_id = orderId

    def historicalData(self, reqId, bar):
        row = pd.DataFrame(
            {"date": [bar.date],
             "open": [bar.open],
             "high": [bar.high],
             "low": [bar.low],
             "close": [bar.close]
             }
        )
        self.historical_data = pd.concat([self.historical_data, row], ignore_index=True)

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        # super().historicalDataEnd(reqId, start, end)
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        self.historical_data_end = reqId


app = ibkr_app()

app.connect('127.0.0.1', 7497, 10645)
while not app.isConnected():
    time.sleep(0.5)


def run_loop():
    app.run()


# Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

while isinstance(app.next_valid_id, type(None)):
    time.sleep(0.01)


value = "EUR.USD"  # This is what your text input looks like on your app
# Create a contract object
contract = Contract()
contract.symbol = value.split(".")[0]
contract.secType = 'CASH'
contract.exchange = 'IDEALPRO'  # 'IDEALPRO' is the currency exchange.
contract.currency = value.split(".")[1]

tickerId = app.next_valid_id

app.reqHistoricalData(
    tickerId, contract, endDateTime='', durationStr='30 D', barSizeSetting='1 hour', whatToShow="MIDPOINT", useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])

while app.historical_data_end != tickerId:
    time.sleep(0.01)

app.disconnect()

app.historical_data


fig = go.Figure(
    data=[
        go.Candlestick(
            x=app.historical_data['date'],
            open=app.historical_data['open'],
            high=app.historical_data['high'],
            low=app.historical_data['low'],
            close=app.historical_data['close']
        )
    ]
)

fig

Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:cashhmds
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefnj
HistoricalDataEnd. ReqId: 1 from 20220122  18:37:50 to 20220221  18:37:50
